In [1]:
## eval
import numpy as np
import oneflow as flow
from typing import Tuple
import oneflow.typing as tp

In [2]:
BATCH_SIZE = 100

In [3]:
def lenet(data, train=False):
    initializer = flow.truncated_normal(0.1)
    conv1 = flow.layers.conv2d(
        data,
        32,
        5,
        padding="SAME",
        activation=flow.nn.relu,
        name="conv1",
        kernel_initializer=initializer,
    )
    pool1 = flow.nn.max_pool2d(
        conv1, ksize=2, strides=2, padding="SAME", name="pool1", data_format="NCHW"
    )
    conv2 = flow.layers.conv2d(
        pool1,
        64,
        5,
        padding="SAME",
        activation=flow.nn.relu,
        name="conv2",
        kernel_initializer=initializer,
    )
    pool2 = flow.nn.max_pool2d(
        conv2, ksize=2, strides=2, padding="SAME", name="pool2", data_format="NCHW"
    )
    reshape = flow.reshape(pool2, [pool2.shape[0], -1])
    hidden = flow.layers.dense(
        reshape,
        512,
        activation=flow.nn.relu,
        kernel_initializer=initializer,
        name="dense1",
    )
    if train:
        hidden = flow.nn.dropout(hidden, rate=0.5, name="dropout")
    return flow.layers.dense(hidden, 10, kernel_initializer=initializer, name="dense2")

In [4]:
##预测
@flow.global_function(type="predict")
def eval_job(
    images: tp.Numpy.Placeholder((BATCH_SIZE, 1, 28, 28), dtype=flow.float),
    labels: tp.Numpy.Placeholder((BATCH_SIZE,), dtype=flow.int32),
) -> Tuple[tp.Numpy, tp.Numpy]:
    with flow.scope.placement("gpu", "0:0"):
        logits = lenet(images, train=False)
        loss = flow.nn.sparse_softmax_cross_entropy_with_logits(
            labels, logits, name="softmax_loss"
        )

    return (labels, logits)

In [5]:
g_total = 0
g_correct = 0

In [6]:
def acc(labels, logits):
    global g_total
    global g_correct

    predictions = np.argmax(logits, 1)
    right_count = np.sum(predictions == labels)
    g_total += labels.shape[0]
    g_correct += right_count

In [7]:
check_point = flow.train.CheckPoint()
check_point.load("./model/lenet_models_1")

In [8]:
(train_images, train_labels), (test_images, test_labels) = flow.data.load_mnist(
        BATCH_SIZE, BATCH_SIZE
    )

./mnist.npz:   0%|          | 50/11221.4296875 [00:00<00:22, 499.93k/s]

File size: 11221.4297 kb, downloading...


./mnist.npz: 11222k [00:00, 16575.38k/s]                                  


Done!
File mnist.npz already exist, path: ./mnist.npz


In [9]:
for epoch in range(1):
    for i, (images, labels) in enumerate(zip(test_images, test_labels)):
        labels, logits = eval_job(images, labels)
        acc(labels, logits)


In [10]:
 print("accuracy: {0:.1f}%".format(g_correct * 100 / g_total))

accuracy: 99.2%
